<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 # Importar librerías
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
from google.colab import drive
import itertools
import math
import logging
import re
import random
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

In [2]:
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Clonar el repositorio de GitHub
!git clone https://github.com/mrk1992/mask-rcnn-tf2-us

# Navegar al directorio del repositorio
%cd Mask-R-CNN-using-Tensorflow2

Cloning into 'mask-rcnn-tf2-us'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 126 (delta 7), reused 6 (delta 6), pack-reused 117
Receiving objects: 100% (126/126), 73.92 MiB | 15.05 MiB/s, done.
Resolving deltas: 100% (17/17), done.
Updating files: 100% (93/93), done.


In [ ]:
# Instalar las dependencias requeridas
!pip install -r requirements.txt

In [21]:
# Numpy con bool da problemas.
np.bool = np.bool_

# Importar Mask RCNN
sys.path.append(os.path.abspath('./'))  # Para encontrar la versión local de la librería
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Ruta a los pesos pre-entrenados de COCO
COCO_WEIGHTS_PATH = "/content/drive/MyDrive/TFM/mask_rcnn_coco.h5"

# Directorio para guardar logs y checkpoints del modelo
DEFAULT_LOGS_DIR = "/content/drive/MyDrive/TFM/logs"

In [22]:
############################################################
#  Configuraciones
############################################################

class PolipoConfig(Config):
    """Configuración para entrenar con el dataset personalizado de pólipos."""
    NAME = "polipo"
    IMAGES_PER_GPU = 2
    NUM_CLASSES = 1 + 1  # Fondo + pólipo
    STEPS_PER_EPOCH = 100
    DETECTION_MIN_CONFIDENCE = 0.9

In [23]:
############################################################
#  Dataset
############################################################

class PolipoDataset(utils.Dataset):
    def load_polipo(self, dataset_dir, subset):
        """Cargar un subconjunto del dataset de pólipos."""
        self.add_class("polipo", 1, "polipo")
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations = list(annotations.values())
        annotations = [a for a in annotations if a['regions']]

        for a in annotations:
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "polipo",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        image_info = self.image_info[image_id]
        if image_info["source"] != "polipo":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        if info["source"] == "polipo":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


# Copiamos el dataset como nos nteresa en el directorio raiz.

In [25]:
import os
import shutil

def create_directory_structure(base_dir):
    subdirs = ['train', 'val', 'test']
    for subdir in subdirs:
        for category in ['images', 'labels']:
            os.makedirs(os.path.join(base_dir, subdir, category), exist_ok=True)

def copy_files(image_source_dir, label_source_dir, dest_dir):
    # Copiar imágenes
    image_files = [f for f in os.listdir(image_source_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    for image_file in image_files:
        shutil.copy(os.path.join(image_source_dir, image_file), os.path.join(dest_dir, 'images', image_file))

    # Copiar etiquetas
    label_files = [f for f in os.listdir(label_source_dir) if f.endswith(('.json', '.xml', '.txt'))]
    for label_file in label_files:
        shutil.copy(os.path.join(label_source_dir, label_file), os.path.join(dest_dir, 'labels', label_file))

# Directorios de origen
source_base_dir = '/content/drive/MyDrive/TFM/dataset_MaskRCNN'
image_source_dirs = {
    'train': os.path.join(source_base_dir, 'train/images'),
    'val': os.path.join(source_base_dir, 'val/images'),
    'test': os.path.join(source_base_dir, 'test/images')
}
label_source_dirs = {
    'train': os.path.join(source_base_dir, 'train/RCNN_labels_merged'),
    'val': os.path.join(source_base_dir, 'val/RCNN_labels_merged'),
    'test': os.path.join(source_base_dir, 'test/RCNN_labels_merged')
}

# Directorio de destino
dest_base_dir = '/content/CurrentDataset'

# Crear estructura de directorios
create_directory_structure(dest_base_dir)

# Copiar archivos a las nuevas ubicaciones
for subset in ['train', 'val', 'test']:
    copy_files(image_source_dirs[subset], label_source_dirs[subset], os.path.join(dest_base_dir, subset))

print("Archivos copiados exitosamente.")


Archivos copiados exitosamente.


In [26]:
############################################################
#  Entrenamiento
############################################################

def train(model):
    """Entrenar el modelo."""
    dataset_train = PolipoDataset()
    dataset_train.load_polyp("/content/CurrentDataset", "train")
    dataset_train.prepare()

    dataset_val = PolipoDataset()
    dataset_val.load_polyp("/content/CurrentDataset", "val")
    dataset_val.prepare()

    print("Entrenando las cabezas de la red")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')

In [29]:
import tensorflow as tf

class AnchorsLayer(tf.keras.layers.Layer):
    def __init__(self, num_anchors, **kwargs):
        super(AnchorsLayer, self).__init__(**kwargs)
        self.num_anchors = num_anchors

    def build(self, input_shape):
        # Implementación para construir las variables necesarias
        self.anchors = self.add_weight(name='anchors',
                                       shape=(2, self.num_anchors, 4),
                                       initializer='uniform',
                                       trainable=True)
        super(AnchorsLayer, self).build(input_shape)

    def call(self, inputs, **kwargs):
        # Implementación para usar las variables creadas
        return self.anchors

# Ejemplo de uso en tu modelo MaskRCNN
anchors = AnchorsLayer(num_anchors=261888)(inputs)


NameError: name 'inputs' is not defined

In [28]:
# Importar los módulos necesarios
import os
import argparse

# Definir las variables simuladas (sustituir con los valores deseados)
args_command = "train"  # Opciones: "train" o "splash"
args_dataset = "/path/to/polyp/dataset/"  # Ruta al dataset de pólipos
args_weights = "/path/to/weights.h5"  # Ruta a los pesos .h5 o 'coco'
args_logs = "/path/to/logs/"  # Directorio de logs y checkpoints
args_image = "path or URL to image"  # Ruta o URL de la imagen
args_video = "path or URL to video"  # Ruta o URL del video

# Configuración de los argumentos simulados
parser = argparse.ArgumentParser(
    description='Entrenar Mask R-CNN para detectar pólipos.')
parser.add_argument("command",
                    metavar="<command>",
                    help="'train' o 'splash'")
parser.add_argument('--dataset', required=False,
                    metavar="/path/to/polyp/dataset/",
                    help='Directorio del dataset de pólipos')
parser.add_argument('--weights', required=True,
                    metavar="/path/to/weights.h5",
                    help="Ruta a los pesos .h5 o 'coco'")
parser.add_argument('--logs', required=False,
                    default=DEFAULT_LOGS_DIR,
                    metavar="/path/to/logs/",
                    help='Directorio de logs y checkpoints (default=logs/)')
parser.add_argument('--image', required=False,
                    metavar="path or URL to image",
                    help='Imagen para aplicar el efecto color splash')
parser.add_argument('--video', required=False,
                    metavar="path or URL to video",
                    help='Video para aplicar el efecto color splash')
args = parser.parse_args([
    args_command,
    '--dataset', args_dataset,
    '--weights', args_weights,
    '--logs', args_logs,
    '--image', args_image,
    '--video', args_video
])

# Validar argumentos
if args.command == "train":
    assert args.dataset, "El argumento --dataset es requerido para entrenar"
elif args.command == "splash":
    assert args.image or args.video, "Proveer --image o --video para aplicar color splash"

print("Pesos: ", args.weights)
print("Dataset: ", args.dataset)
print("Logs: ", args.logs)

# Configuraciones (suponemos que PolypConfig y modellib ya están definidos en otro lugar)
if args.command == "train":
    config = PolipoConfig()
else:
    class InferenceConfig(PolipoConfig):
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    config = InferenceConfig()
config.display()

# Crear el modelo
if args.command == "train":
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=args.logs)
else:
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=args.logs)

# Seleccionar el archivo de pesos a cargar
if args.weights.lower() == "coco":
    weights_path = COCO_WEIGHTS_PATH
    if not os.path.exists(weights_path):
        utils.download_trained_weights(weights_path)
elif args.weights.lower() == "last":
    weights_path = model.find_last()
elif args.weights.lower() == "imagenet":
    weights_path = model.get_imagenet_weights()
else:
    weights_path = args.weights

# Cargar los pesos
print("Cargando pesos ", weights_path)
if args.weights.lower() == "coco":
    model.load_weights(weights_path, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc",
        "mrcnn_bbox", "mrcnn_mask"])
else:
    model.load_weights(weights_path, by_name=True)

# Entrenar o evaluar
if args.command == "train":
    train(model)
elif args.command == "splash":
    detect_and_color_splash(model, image_path=args.image,
                            video_path=args.video)
else:
    print("'{}' no es reconocido. Use 'train' o 'splash'".format(args.command))



Pesos:  /path/to/weights.h5
Dataset:  /path/to/polyp/dataset/
Logs:  /path/to/logs/

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss'

ValueError: Exception encountered when calling layer "anchors" (type Lambda).


The following Variables were created within a Lambda layer (anchors)
but are not tracked by said layer:
  <tf.Variable 'anchors/Variable:0' shape=(2, 261888, 4) dtype=float32>
The layer cannot safely ensure proper Variable reuse across multiple
calls, and consequently this behavior is disallowed for safety. Lambda
layers are not well suited to stateful computation; instead, writing a
subclassed Layer is the recommend way to define layers with
Variables.

Call arguments received by layer "anchors" (type Lambda):
  • inputs=tf.Tensor(shape=(None, None, None, 3), dtype=float32)
  • mask=None
  • training=None